In [1]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from statistics import mean
import pandas as pd

import torch as th

from tabpfn import TabPFNClassifier

from tab_pfn.networks import SCM, TabPFN
from tab_pfn.metrics import AccuracyMeter

import warnings

warnings.filterwarnings("ignore")

In [17]:
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

my_tab_pfn = TabPFN(100, 10, 256, 512, 1024, 4, 12)
my_tab_pfn.load_state_dict(th.load("/home/samuel/PycharmProjects/TabPFN/out_2/model_48127.pt"))
my_tab_pfn.cuda()
my_tab_pfn.eval()

n_datasets = 128

accuracy_meter_original = AccuracyMeter(None)
accuracy_meter_our = AccuracyMeter(None)
nb = 0

for _ in tqdm(range(n_datasets)):
    scm = SCM(100, (2, 10), False)
    x, y = scm(1024)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    
    try:
        classifier.fit(x_train, y_train)
        p_eval = th.tensor(classifier.predict_proba(x_test))[None]
        
        accuracy_meter_original.add(p_eval.cpu(), y_test[None])
        
        x_train = x_train[None].cuda()
        y_train = y_train[None].cuda()
        x_test = x_test[None].cuda()
        y_test = y_test[None]
        
        p_eval = my_tab_pfn(x_train, y_train, x_test)
        
        accuracy_meter_our.add(p_eval.cpu(), y_test.cpu())
    except ValueError:
        pass

  0%|          | 0/128 [00:00<?, ?it/s]/home/samuel/PycharmProjects/TabPFN/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  1%|          | 1/128 [00:00<01:04,  1.96it/s]/home/samuel/PycharmProjects/TabPFN/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the

In [18]:
print("original", accuracy_meter_original.accuracy())
print("our", accuracy_meter_our.accuracy())

original 0.7339607696966873
our 0.7431160601966175


In [24]:
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

df = pd.read_csv("/home/samuel/Téléchargements/mfeat-karhunen_zip/data/mfeat-karhunen_csv.csv", sep=",")

x = df.iloc[:, :-1]
y = df.iloc[:, -1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

classifier.fit(x_train, y_train - 1)
p_eval = classifier.predict_proba(x_test)

accuracy_meter = AccuracyMeter(None)
accuracy_meter.add(th.tensor(p_eval), th.tensor(y_test.tolist()) - 1)

/home/samuel/PycharmProjects/TabPFN/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [25]:
accuracy_meter.accuracy()

0.966

In [22]:
p_eval.shape

(1000, 10)

In [23]:
th.tensor(y_test.tolist())

tensor([10,  2,  7,  5,  7,  7,  5,  9,  1,  7,  1,  7,  6,  3,  2,  2,  4,  1,
         9, 10,  4,  9, 10,  6, 10,  1,  6,  5,  4,  1,  4,  5,  3,  5,  6,  3,
         9,  7,  6,  2,  1, 10,  2,  3,  8,  5,  3,  5,  1,  9,  8,  2,  8,  2,
         5,  9,  7,  9,  6,  2,  8,  4,  9,  6, 10, 10,  5,  8,  1,  8,  8,  2,
         7,  6,  4,  2,  9,  4,  2, 10,  1,  5,  8,  8,  9,  8,  9,  3,  2,  9,
         8,  9,  2,  2, 10, 10,  4,  7,  8, 10,  5,  5,  3,  7,  2,  5,  4,  2,
         4,  3,  4,  8,  6,  9,  4,  2,  6,  5,  2,  9, 10,  6,  6,  3,  2,  8,
         3,  5,  6,  2, 10, 10,  4,  3,  5,  7, 10, 10,  9,  8,  7,  2,  9,  5,
         8,  3,  4,  7,  2,  1,  9,  5,  4,  6,  2,  1,  3,  9,  3,  9,  4,  5,
         1,  6, 10,  8,  9,  7,  2,  9,  6,  4, 10, 10,  3,  7,  4,  7,  5,  9,
         7,  4, 10,  6,  5,  4, 10,  9,  6,  7,  4,  2,  7,  2,  1,  3,  6,  5,
         4,  8, 10,  3,  1,  4,  3,  2,  8,  7,  6,  1,  9,  1,  2,  1,  4, 10,
         6,  2,  3,  7,  5,  1,  3,  6, 